<a href="https://colab.research.google.com/github/rrwiren/ilmanlaatu-ennuste-helsinki/blob/main/GRU_pilkottu_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 0. Esitiedot ja Tavoite (GRU v3 - Uusi Data)
"""
Otsoniennuste Helsinki - GRU-malli v3 (Uudella Datalla)

Tavoite:
1. Ladata uusi esikäsitelty data Parquet-tiedostosta, joka sisältää nyt
   myös pilvisyyden ('processed_Helsinki_O3_Weather_Cloudiness_2024_2025_v3.parquet').
2. Esikäsitellä data neuroverkkomallille sopivaksi:
   - Ominaisuuksien muokkaus (sykliset aika- ja tuulipiirteet).
   - Datan jako harjoitus-, validointi- ja testijoukkoihin.
   - Datan skaalaus.
   - Sekvenssien luonti.
3. Määritellä ja kouluttaa GRU (Gated Recurrent Unit) -malli PyTorchilla.
4. Tehdä ennusteita testijaksolle (seuraavat 24 tuntia).
5. Arvioida mallin suorituskykyä monipuolisesti (RMSE, MAE, baseline-vertailu,
   8h varoitusrajan metriikat) ja analysoida tuloksia.
6. Visualisoida ennusteita.
"""
print("Osa 0: Esitiedot (GRU v3) - OK")

In [ ]:
# @title 1. Tuonnit ja Asetukset (GRU v3)

# Peruskirjastot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import os
import math
import copy
import traceback
import warnings

# Sklearn esikäsittelyyn ja metriikoihin
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report

# PyTorch kirjastot
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Edistymispalkki
from tqdm.notebook import tqdm

# --- Data-asetukset ---
BASE_GITHUB_URL = 'https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/'
# *** TÄRKEÄÄ: Käytetään uutta Parquet-tiedostoa ***
PARQUET_PATH = 'data/processed/processed_Helsinki_O3_Weather_Cloudiness_2024_2025_v3.parquet'
DATA_URL = BASE_GITHUB_URL + PARQUET_PATH
LOCAL_DATA_PATH = 'processed_Helsinki_O3_Weather_Cloudiness_2024_2025_v3.parquet' # Paikallinen nimi lataukselle

# --- Sarakemääritykset ---
TARGET_COLUMN = 'Otsoni [µg/m³]'

# Sarakkeet, jotka luetaan Parquet-tiedostosta ja käytetään *ennen* feature engineeringiä
# Varmista, että nämä vastaavat Parquet-tiedoston sarakkeita
ALL_COLUMNS_IN_PARQUET = [
    'Otsoni [µg/m³]',
    'Lämpötilan keskiarvo [°C]',
    'Keskituulen nopeus [m/s]',
    'Ilmanpaineen keskiarvo [hPa]',
    'Tuulen suunnan keskiarvo [°]',
    'Pilvisyys [okta]'
]
# Lopulliset ominaisuudet mallille määritetään feature engineeringin jälkeen

# --- Mallinnusasetukset ---
# Aikasarjaparametrit
SEQUENCE_LENGTH = 72      # Kuinka monta tuntia historiaa syötteenä (3 vrk)
PREDICTION_HORIZON = 24 # Kuinka monta tuntia ennustetaan

# GRU-mallin Hyperparametrit
# INPUT_SIZE lasketaan myöhemmin feature engineeringin jälkeen
HIDDEN_SIZE = 64
NUM_LAYERS = 2
OUTPUT_SIZE = PREDICTION_HORIZON # Ennustetaan 24h kerralla
DROPOUT_PROB = 0.2

# --- Koulutusparametrit ---
BATCH_SIZE = 64
LEARNING_RATE = 0.001
EPOCHS = 75  # Kasvatetaan hieman epochien määrää neuroverkolle
TEST_SPLIT_RATIO = 0.15
VALID_SPLIT_RATIO = 0.15
EARLY_STOPPING_PATIENCE = 10 # Kasvatetaan myös kärsivällisyyttä

# --- Varoitusraja (8h liukuva keskiarvo) ---
# Määritellään tässä, ja voidaan määrittää uudelleen arviointisolussa varmuuden vuoksi
O3_THRESHOLD_8H_AVG = 85 # µg/m³ (tai esim. 85 testailuun)

# --- Yleisasetukset ---
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Asetukset kuvaajille
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 7) # Hieman isommat kuvaajat
# Estetään turhia varoituksia
warnings.filterwarnings("ignore")


if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print(f"Osa 1: Tuonnit ja Asetukset (GRU v3) - OK")
print(f"Käytettävä laite: {device}")
print(f"Ladataan data: {DATA_URL}")
print(f"Kohdemuuttuja: {TARGET_COLUMN}")

In [ ]:
# @title 2. Funktiot Datan Lataukseen ja Käsittelyyn (GRU v3)

import pandas as pd
import numpy as np
import requests
import io
import os
import re
import traceback

# Funktio datan lataamiseen URL:sta (jos paikallista ei löydy)
def download_data(url, local_path):
    """Lataa tiedoston URL:sta paikalliseen polkuun."""
    try:
        print(f"Yritetään ladata dataa osoitteesta {url}...")
        response = requests.get(url)
        response.raise_for_status()
        # Varmista että hakemisto on olemassa
        # Tarkista onko local_path tiedosto vai hakemisto
        target_dir = os.path.dirname(local_path)
        if target_dir and not os.path.exists(target_dir): # Luo hakemisto vain jos se ei ole tyhjä (eli ei juurihakemisto)
             os.makedirs(target_dir, exist_ok=True)
             print(f"Luotiin hakemisto: {target_dir}")

        with open(local_path, 'wb') as f:
            f.write(response.content)
        print(f"Data ladattu onnistuneesti: {local_path}")
        return True
    except requests.exceptions.RequestException as e:
        print(f"Virhe datan latauksessa: {e}")
        return False
    except Exception as e:
         print(f"Odottamaton virhe tiedoston tallennuksessa polkuun {local_path}: {e}")
         traceback.print_exc()
         return False

# Funktio Parquet-datan lataamiseen
def load_parquet_data(filepath_or_url, local_cache_path=LOCAL_DATA_PATH):
    """Lataa datan Parquet-tiedostosta (paikallisesti tai URL:sta, käyttää välimuistia)."""
    filepath_to_read = None
    # Jos annettu on URL
    if filepath_or_url.startswith('http'):
         # Tarkista ensin paikallinen välimuisti
         if os.path.exists(local_cache_path):
              print(f"Käytetään paikallista välimuistitiedostoa: {local_cache_path}")
              filepath_to_read = local_cache_path
         # Jos ei välimuistissa, yritä ladata
         else:
              print(f"Paikallista tiedostoa {local_cache_path} ei löytynyt.")
              if download_data(filepath_or_url, local_cache_path):
                   filepath_to_read = local_cache_path
              else:
                   return None # Lataus epäonnistui
    # Jos annettu on paikallinen polku
    else:
         filepath_to_read = filepath_or_url
         if not os.path.exists(filepath_to_read):
              print(f"VIRHE: Annettua paikallista tiedostoa {filepath_to_read} ei löytynyt.")
              return None

    # Jos tiedostopolku on saatu (joko paikallisesti tai lataamalla)
    if filepath_to_read:
        try:
            print(f"Ladataan dataa tiedostosta: {filepath_to_read}")
            df = pd.read_parquet(filepath_to_read)
            print(f"Parquet-tiedosto ladattu, muoto: {df.shape}")

            # Varmista DatetimeIndex
            if not isinstance(df.index, pd.DatetimeIndex):
                print("Indeksi ei ole DatetimeIndex. Yritetään muuntaa...")
                try:
                    df.index = pd.to_datetime(df.index)
                    print("Indeksi muunnettu datetime-indeksiksi.")
                except Exception as e_idx:
                    print(f"VIRHE: Indeksin muunto datetimeksi epäonnistui: {e_idx}")
                    return None # Palauta None jos kriittinen virhe

            # Varmista aikavyöhyke (jos puuttuu, aseta Europe/Helsinki)
            if df.index.tz is None:
                 print("Asetetaan aikavyöhykkeeksi Europe/Helsinki...")
                 try:
                     df = df.tz_localize('Europe/Helsinki', ambiguous='NaT', nonexistent='NaT')
                     rows_before_nat = len(df)
                     # Poista rivit, jotka saivat NaT-aikaleiman DST-käsittelyssä
                     df.dropna(axis=0, subset=[df.index.name], inplace=True)
                     if len(df) < rows_before_nat:
                          print(f"Poistettu {rows_before_nat - len(df)} riviä aikavyöhykkeen asetuksen (NaT) vuoksi.")
                 except Exception as e_tz:
                      print(f"VAROITUS: Aikavyöhykkeen asetus epäonnistui: {e_tz}. Jatketaan naiivilla indeksillä.")

            df.sort_index(inplace=True)
            print(f"Datan perustiedot: Aikaväli=[{df.index.min()} - {df.index.max()}]")
            print(f"Sarakkeet: {df.columns.tolist()}")

            # Tarkista NaN-arvot
            if df.isnull().any().any():
                print("VAROITUS: Datassa NaN-arvoja latauksen jälkeen. Yritetään täyttää ffill/bfill...")
                df.ffill(inplace=True)
                df.bfill(inplace=True)
                if df.isnull().any().any():
                     print("VIRHE: Ei voitu täyttää kaikkia NaN-arvoja. Poistetaan rivit joissa NaN.")
                     df.dropna(inplace=True)
                     print(f"Datan muoto NaN-poiston jälkeen: {df.shape}")

            return df

        except Exception as e:
            print(f"VIRHE Parquet-tiedoston lukemisessa ('{filepath_to_read}'): {e}")
            traceback.print_exc()
            return None
    else:
         # Tänne ei pitäisi päätyä jos logiikka toimii, mutta varmistus
         print("VIRHE: Tiedostopolkua lukemiseen ei saatu määritettyä.")
         return None


# Funktio ominaisuuksien muokkaukseen (feature engineering) neuroverkolle
def feature_engineer_gru(df):
    """Lisää aika- ja syklisiä ominaisuuksia neuroverkkokäyttöön."""
    print("\nSuoritetaan ominaisuuksien muokkaus (Feature Engineering)...")
    if not isinstance(df, pd.DataFrame):
        print("VIRHE: feature_engineer_gru sai syötteenä jotain muuta kuin DataFramen.")
        return None
    df_eng = df.copy()

    # Varmistetaan, että indeksi on DatetimeIndex
    if not isinstance(df_eng.index, pd.DatetimeIndex):
        print("VIRHE: DataFrame-indeksi ei ole DatetimeIndex feature engineeringissä.")
        return df_eng # Palauta alkuperäinen, jos indeksi väärä

    # 1. Kellonaika (syklinen)
    try:
        df_eng['hour_sin'] = np.sin(2 * np.pi * df_eng.index.hour / 24.0)
        df_eng['hour_cos'] = np.cos(2 * np.pi * df_eng.index.hour / 24.0)
        print("Lisätty syklinen kellonaika (hour_sin, hour_cos).")
    except Exception as e: print(f"VIRHE kellonaikaominaisuuksien luonnissa: {e}")

    # 2. Tuulen suunta (syklinen)
    # Käytetään Osa 1:ssä määriteltyä sarakkeen nimeä (oletetaan sen olevan ORIGINAL_FEATURE_COLUMNSissa)
    wind_dir_col_orig = 'Tuulen suunnan keskiarvo [°]'
    if wind_dir_col_orig in df_eng.columns:
        try:
            # Muunnetaan numeeriseksi ensin, jos se ei ole (esim. object-tyyppiä)
            df_eng[wind_dir_col_orig] = pd.to_numeric(df_eng[wind_dir_col_orig], errors='coerce')
            # Tarkista tuliko NaNneja
            if df_eng[wind_dir_col_orig].isnull().any():
                 print(f"VAROITUS: NaN-arvoja sarakkeessa '{wind_dir_col_orig}' tyyppimuunnoksen jälkeen. Täytetään...")
                 df_eng[wind_dir_col_orig].ffill(inplace=True)
                 df_eng[wind_dir_col_orig].bfill(inplace=True)
                 df_eng[wind_dir_col_orig].dropna(inplace=True) # Poista jos jäi vielä

            df_eng['wind_dir_rad'] = np.deg2rad(df_eng[wind_dir_col_orig])
            df_eng['wind_dir_sin'] = np.sin(df_eng['wind_dir_rad'])
            df_eng['wind_dir_cos'] = np.cos(df_eng['wind_dir_rad'])
            # Poista alkuperäinen tuulensuunta asteina ja radiaaneina
            df_eng.drop(columns=[wind_dir_col_orig, 'wind_dir_rad'], inplace=True, errors='ignore')
            print("Muunnettu tuulen suunta sykliseksi (wind_dir_sin, wind_dir_cos).")
        except Exception as e:
            print(f"VIRHE tuulensuuntaominaisuuksien luonnissa: {e}")
            # Poista mahdolliset osittain luodut sarakkeet
            df_eng.drop(columns=['wind_dir_rad', 'wind_dir_sin', 'wind_dir_cos'], inplace=True, errors='ignore')
    else:
        print(f"Saraketta '{wind_dir_col_orig}' ei löytynyt, tuulensuunnan muokkausta ei tehty.")

    # 3. Muita mahdollisia (esim. kuukausi, viikonpäivä) - jätetään nyt pois
    # ...

    print("Ominaisuuksien muokkaus valmis.")
    # Palautetaan DataFrame, jossa on alkuperäiset + uudet sarakkeet (miinus poistetut)
    return df_eng

# Funktio sekvenssien luontiin PyTorchille
def create_sequences_pytorch(features_scaled, targets_original, sequence_length, prediction_horizon):
    """
    Luo syötesekvenssejä (X, skaalattu) ja alkuperäisiä kohde-ennusteita (y_orig).
    Kohde (y_orig) sisältää vain TARGET_COLUMNin arvot.
    Palauttaa X ja y_orig numpy arrayna.
    """
    X, y_orig = [], []
    print(f"\nLuodaan PyTorch-sekvenssejä: sequence_length={sequence_length}, prediction_horizon={prediction_horizon}")
    print(f"features_scaled shape: {features_scaled.shape}, targets_original shape: {targets_original.shape}")

    required_len = sequence_length + prediction_horizon
    if len(features_scaled) < required_len:
        print(f"VAROITUS: Ei tarpeeksi dataa ({len(features_scaled)}) sekvenssien luomiseen. Tarvitaan vähintään {required_len}.")
        return np.array(X), np.array(y_orig)

    # Varmistetaan targets_original muoto (N, 1) käsittelyä varten
    if targets_original.ndim == 1:
        targets_original = targets_original.reshape(-1, 1)
        print(f"Muutettu targets_original muotoon: {targets_original.shape}")

    for i in range(len(features_scaled) - required_len + 1):
        # X sisältää kaikki skaalatut ominaisuudet menneisyydestä
        X.append(features_scaled[i:(i + sequence_length)])
        # y sisältää VAIN kohdemuuttujan arvot tulevaisuudesta (alkuperäisessä skaalassa)
        # Otetaan [:, 0], jotta saadaan 1D array (horizon,) ennen kuin lisätään listaan
        y_orig.append(targets_original[i + sequence_length : i + sequence_length + prediction_horizon, 0])

    print(f"Luotu {len(X)} sekvenssiä.")
    X = np.array(X)     # Muoto: (samples, sequence_length, num_features)
    y_orig = np.array(y_orig) # Muoto: (samples, prediction_horizon)

    # Muutetaan y_orig muotoon (samples, prediction_horizon, 1) PyTorchia varten
    if y_orig.ndim == 2:
         y_orig = y_orig[..., np.newaxis]
         print(f"Muutettu y_orig muotoon: {y_orig.shape}")
    elif y_orig.size > 0: # Varmista ettei yritetä muokata tyhjää arrayta
         print(f"VAROITUS: y_orig muoto ({y_orig.shape}) ei ole odotettu (samples, horizon) tai (samples, horizon, 1).")

    return X, y_orig

print("Osa 2: Funktiot datan käsittelyyn (GRU v3) - OK")

In [ ]:
# @title 3. Pääskriptin Suoritus: Datan Käsittely (GRU v3)

import pandas as pd
import numpy as np
import traceback
# Varmistetaan, että PyTorch on tuotu
try:
    import torch
    from torch.utils.data import TensorDataset, DataLoader
except ImportError:
    print("VIRHE: PyTorch ei ole asennettu tai sitä ei voitu tuoda.")
    # Aseta loaderit Noneksi, jotta myöhemmät vaiheet eivät suoritu
    train_loader = None; valid_loader = None; test_loader = None;
# Varmistetaan Scaler
try:
    from sklearn.preprocessing import StandardScaler
except ImportError:
     print("VIRHE: scikit-learn ei ole asennettu tai sitä ei voitu tuoda.")
     feature_scaler = None; o3_scaler = None

print("--- Aloitetaan Datan Käsittely GRU-mallia varten ---")

# Alustetaan muuttujat varmuuden vuoksi
df_raw_full = None; df_raw = None; df_engineered = None
train_loader = None; valid_loader = None; test_loader = None
feature_scaler = None; o3_scaler = None
INPUT_SIZE = None
test_timestamps = None
# Sekvenssimuuttujat
X_train, y_train_original, X_valid, y_valid_original, X_test, y_test_original = [None]*6
y_train_scaled, y_valid_scaled = None, None


# Suoritetaan päälogiikka try-except-lohkossa
try:
    # 1. Lataa data käyttäen Osa 2:n funktiota
    df_raw_full = load_parquet_data(DATA_URL) # Käyttää DATA_URL ja LOCAL_DATA_PATH Osa 1:stä

    if df_raw_full is not None:
        # 2. Tarkista ja valitse alkuperäiset sarakkeet
        print("\nTarkistetaan ja valitaan alkuperäiset sarakkeet...")
        missing_cols_raw = [col for col in ALL_COLUMNS_IN_PARQUET if col not in df_raw_full.columns]
        if missing_cols_raw:
            raise ValueError(f"Seuraavat määritellyt sarakkeet puuttuvat Parquet-tiedostosta: {missing_cols_raw}")
        df_raw = df_raw_full[ALL_COLUMNS_IN_PARQUET].copy()
        print("Alkuperäiset sarakkeet valittu.")

        # 3. Suorita Feature Engineering
        df_engineered = feature_engineer_gru(df_raw)
        if df_engineered is None: raise ValueError("Ominaisuuksien muokkaus (feature_engineer_gru) epäonnistui.")

        # 4. Määritä lopulliset ominaisuudet ja INPUT_SIZE
        FINAL_FEATURE_COLUMNS = df_engineered.columns.tolist()
        # Varmista, ettei kohdemuuttuja ole vahingossa ominaisuuksissa
        # (Sen pitäisi olla, koska käytämme historiaa, mutta tarkistus on hyvä)
        if TARGET_COLUMN not in FINAL_FEATURE_COLUMNS:
             print(f"VAROITUS: Kohdemuuttuja '{TARGET_COLUMN}' ei ole lopullisissa ominaisuuksissa!")
             # Tässä vaiheessa voitaisiin päättää lisätä se tai pysäyttää ajo
        INPUT_SIZE = len(FINAL_FEATURE_COLUMNS)
        if INPUT_SIZE == 0: raise ValueError("Lopullisia ominaisuuksia ei löytynyt feature engineeringin jälkeen.")
        print(f"\nLopullinen ominaisuuksien määrä (INPUT_SIZE): {INPUT_SIZE}")
        print(f"Lopulliset ominaisuudet mallille: {FINAL_FEATURE_COLUMNS}")

        # 5. Jaa data harjoitus-, validointi- ja testijoukkoihin
        n = len(df_engineered)
        min_data_len_needed = (SEQUENCE_LENGTH + PREDICTION_HORIZON) * 3
        if n < min_data_len_needed:
             print(f"VAROITUS: Datan pituus ({n}) voi olla liian lyhyt jakoon ja sekvenssien luontiin (tarvitaan > {min_data_len_needed}).")

        test_split_idx = int(n * (1 - TEST_SPLIT_RATIO))
        valid_split_idx = int(test_split_idx * (1 - VALID_SPLIT_RATIO / (1 - TEST_SPLIT_RATIO)))

        df_train = df_engineered[:valid_split_idx]
        df_valid = df_engineered[valid_split_idx:test_split_idx]
        df_test = df_engineered[test_split_idx:]

        min_len_for_seq = SEQUENCE_LENGTH + PREDICTION_HORIZON
        if len(df_train) < min_len_for_seq or len(df_valid) < min_len_for_seq or len(df_test) < min_len_for_seq:
            print(f"Train len: {len(df_train)}, Valid len: {len(df_valid)}, Test len: {len(df_test)}, Min required: {min_len_for_seq}")
            raise ValueError("Liian vähän dataa yhdessä tai useammassa jaossa sekvenssien luontia varten.")

        print(f"\nDatan jako:")
        print(f"Train: {df_train.shape[0]} riviä ({df_train.index.min()} - {df_train.index.max()})")
        print(f"Valid: {df_valid.shape[0]} riviä ({df_valid.index.min()} - {df_valid.index.max()})")
        print(f"Test:  {df_test.shape[0]} riviä ({df_test.index.min()} - {df_test.index.max()})")

        # 6. Skaalaa data
        print("\nSkaalataan dataa...")
        # Skaalaa kaikki lopulliset ominaisuudet
        feature_scaler = StandardScaler()
        # Sovita VAIN harjoitusdataan
        scaled_train_features = feature_scaler.fit_transform(df_train)
        # Muunna validointi- ja testidata
        scaled_valid_features = feature_scaler.transform(df_valid)
        scaled_test_features = feature_scaler.transform(df_test)
        print("Ominaisuudet skaalattu.")

        # Luo ja sovita skaalain erikseen VAIN kohdemuuttujalle (käytä df_raw:ta!)
        o3_scaler = StandardScaler()
        o3_scaler.fit(df_raw.loc[df_train.index, [TARGET_COLUMN]])
        print("Erillinen skaalain luotu ja sovitettu kohdemuuttujalle (O3).")

        # 7. Hae alkuperäiset O3-kohdearvot (skaalaamattomat)
        o3_train_targets_original = df_raw.loc[df_train.index, [TARGET_COLUMN]].values
        o3_valid_targets_original = df_raw.loc[df_valid.index, [TARGET_COLUMN]].values
        o3_test_targets_original = df_raw.loc[df_test.index, [TARGET_COLUMN]].values

        # 8. Luo sekvenssit
        X_train, y_train_original = create_sequences_pytorch(scaled_train_features, o3_train_targets_original, SEQUENCE_LENGTH, PREDICTION_HORIZON)
        X_valid, y_valid_original = create_sequences_pytorch(scaled_valid_features, o3_valid_targets_original, SEQUENCE_LENGTH, PREDICTION_HORIZON)
        X_test, y_test_original = create_sequences_pytorch(scaled_test_features, o3_test_targets_original, SEQUENCE_LENGTH, PREDICTION_HORIZON)

        if X_train.size == 0 or X_valid.size == 0 or X_test.size == 0:
             raise ValueError("Sekvenssien luonti epäonnistui (yksi tai useampi X on tyhjä).")

        # 9. Skaalaa kohdesekvenssit (y) koulutusta ja validointia varten
        # Muoto y_..._original on nyt (samples, horizon, 1)
        y_train_scaled = o3_scaler.transform(y_train_original.reshape(-1, 1)).reshape(y_train_original.shape)
        y_valid_scaled = o3_scaler.transform(y_valid_original.reshape(-1, 1)).reshape(y_valid_original.shape)
        print("Kohdesekvenssit (y) skaalattu koulutusta ja validointia varten.")

        # 10. Muunna PyTorch Tensoreiksi
        print("\nMuunnetaan data PyTorch-tensoreiksi...")
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32)
        X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32)
        y_valid_tensor = torch.tensor(y_valid_scaled, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor_original = torch.tensor(y_test_original, dtype=torch.float32) # Alkuperäinen y testiä varten
        print("Tensorit luotu.")

        # 11. Luo DataLoaderit
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True) # drop_last voi auttaa jos viimeinen erä on pieni
        valid_dataset = TensorDataset(X_valid_tensor, y_valid_tensor)
        valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor_original) # Testilataaja tarvitsee X:n ja alkuperäisen y:n
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
        print("DataLoaderit luotu.")

        # 12. Tallenna testiaikaleimat visualisointia varten
        print("\nTallennetaan testiaikaleimoja...")
        # Varmistetaan df_test ja X_test olemassaolo ja pituus
        if 'df_test' in locals() and not df_test.empty and len(df_test) >= SEQUENCE_LENGTH and 'X_test' in locals() and len(X_test) > 0:
            try:
                test_start_index_loc = df_engineered.index.get_loc(df_test.index[0])
                test_start_indices = test_start_index_loc + SEQUENCE_LENGTH
                end_index = test_start_indices + len(X_test) # X_test pituus vastaa sekvenssien määrää
                if end_index <= len(df_engineered.index):
                    test_timestamps = df_engineered.index[test_start_indices : end_index]
                    print(f"Testiaikaleimat tallennettu ({len(test_timestamps)} kpl). Alkaa: {test_timestamps.min()}, Päättyy: {test_timestamps.max()}")
                else:
                    print(f"VAROITUS: Ei voitu määrittää kaikkia testiaikaleimoja, loppuindeksi ({end_index}) ylittää datan pituuden ({len(df_engineered.index)}).")
                    test_timestamps = None
            except KeyError:
                 print("VAROITUS: Testijoukon alkuaikaleimaa ei löytynyt alkuperäisestä indeksistä.")
                 test_timestamps = None
            except Exception as e_ts:
                 print(f"VIRHE testiaikaleimojen tallennuksessa: {e_ts}")
                 test_timestamps = None
        else:
             print("VAROITUS: Testidata liian lyhyt, tyhjä tai sekvenssejä ei luotu. Ei voida tallentaa aikaleimoja.")
             test_timestamps = None


        print(f"\nLopulliset muodot DataLoadereihin menevälle datalle:")
        print(f"X_train_tensor: {X_train_tensor.shape}, y_train_tensor: {y_train_tensor.shape}")
        print(f"X_valid_tensor: {X_valid_tensor.shape}, y_valid_tensor: {y_valid_tensor.shape}")
        print(f"X_test_tensor:  {X_test_tensor.shape}, y_test_tensor_original: {y_test_tensor_original.shape}")

        print("\nOsa 3: Datan käsittely (GRU v3) - VALMIS")

    # else: # df_raw_full is None
    #     print("Datan lataus epäonnistui kokonaan. Skriptin suoritus keskeytetty aiemmin.")

# Käsittele mahdolliset virheet päälohkossa
except ValueError as ve:
     print(f"\n---> VIRHE DATAN KÄSITTELYSSÄ (ValueError): {ve} <---")
     traceback.print_exc()
     # Asetetaan loaderit Noneksi, jotta myöhemmät vaiheet eivät suoritu
     train_loader = None; valid_loader = None; test_loader = None; INPUT_SIZE=None;
except KeyError as ke:
     print(f"\n---> VIRHE DATAN KÄSITTELYSSÄ (KeyError): Saraketta {ke} ei löytynyt <---")
     traceback.print_exc()
     train_loader = None; valid_loader = None; test_loader = None; INPUT_SIZE=None;
except Exception as e:
     print(f"\n---> ODOTTAMATON VIRHE DATAN KÄSITTELYSSÄ (Osa 3) <---")
     print(f"Virhetyyppi: {type(e).__name__}")
     print(f"Virheilmoitus: {e}")
     traceback.print_exc()
     train_loader = None; valid_loader = None; test_loader = None; INPUT_SIZE=None;
     print("----------------------------------------------------")

In [ ]:
# @title 4. GRU-Mallin Määrittely (GRU v3)

import torch
import torch.nn as nn
import traceback # Virheiden jäljitykseen

print("--- Määritellään GRU-malli ---")

# Varmistetaan tarvittavat parametrit Osa 1:stä
required_params = ['INPUT_SIZE', 'HIDDEN_SIZE', 'NUM_LAYERS', 'OUTPUT_SIZE', 'DROPOUT_PROB']
params_ok = True
for p in required_params:
    if p not in locals() or locals()[p] is None:
         print(f"VIRHE: Mallin hyperparametri '{p}' puuttuu tai on None. Aja Osa 1 uudelleen.")
         params_ok = False
if not params_ok:
     raise ValueError("Yksi tai useampi mallin hyperparametri puuttuu.")

# Itse malliluokka
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob):
        super(GRUModel, self).__init__()
        # Varmistetaan parametrien oikeellisuus myös tässä
        assert isinstance(input_size, int) and input_size > 0, "input_size > 0"
        assert isinstance(hidden_size, int) and hidden_size > 0, "hidden_size > 0"
        assert isinstance(num_layers, int) and num_layers > 0, "num_layers > 0"
        assert isinstance(output_size, int) and output_size > 0, "output_size > 0"
        assert isinstance(dropout_prob, float) and 0.0 <= dropout_prob < 1.0, "dropout_prob [0, 1)"

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # GRU-kerros
        # batch_first=True -> input/output muoto: (batch, seq_len, features)
        # dropout lisätään vain jos kerroksia > 1
        gru_dropout = dropout_prob if num_layers > 1 else 0.0
        self.gru = nn.GRU(input_size, hidden_size, num_layers,
                          batch_first=True, dropout=gru_dropout)

        # Lineaarinen kerros, joka muuntaa GRU:n piilotilan ennusteeksi
        # Sisääntulo: hidden_size, Ulostulo: output_size (esim. 24 tuntia)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x:n odotettu muoto: (batch_size, sequence_length, input_size)

        # Alustetaan piilotila nollilla
        # Muoto: (num_layers, batch_size, hidden_size)
        # Siirretään piilotila samalle laitteelle kuin input x
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Eteenpäinsyöttö GRU-kerroksen läpi
        # out: sisältää kaikkien aika-askelten piilotilat viimeisestä kerroksesta
        #      muoto: (batch_size, sequence_length, hidden_size)
        out, _ = self.gru(x, h0)

        # Otetaan vain viimeisen aika-askeleen piilotila (-1) GRU:n ulostulosta
        # Muoto: (batch_size, hidden_size)
        out = out[:, -1, :]

        # Syötetään viimeinen piilotila lineaarisen kerroksen läpi
        # Muoto: (batch_size, output_size)
        out = self.fc(out)
        return out

# Testataan luokan alustus nopeasti (varmistaa että parametrit ok)
try:
     # Luo malli testiksi käyttäen Osa 1:n parametreja
     print(f"\nTestataan mallin alustusta: INPUT={INPUT_SIZE}, HIDDEN={HIDDEN_SIZE}, LAYERS={NUM_LAYERS}, OUTPUT={OUTPUT_SIZE}, DROPOUT={DROPOUT_PROB}")
     temp_model_test = GRUModel(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, OUTPUT_SIZE, DROPOUT_PROB)
     print("GRUModel-luokka määritelty ja alustus testattu onnistuneesti.")
     # Tulostetaan mallin rakenne
     print("\nMallin rakenne:")
     print(temp_model_test)
     # Vapauta muisti heti
     del temp_model_test
except Exception as e_model_def:
     print(f"VIRHE GRUModel-luokan alustuksessa: {e_model_def}")
     traceback.print_exc()


print("\nOsa 4: GRU-Mallin Määrittely (GRU v3) - OK")

In [ ]:
# @title 5. Koulutusfunktion Määrittely (GRU v3)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm # Edistymispalkki
import copy # Mallin tilan kopiointiin
import traceback

print("--- Määritellään train_model -funktio ---")

# Varmistetaan tarvittavien luokkien olemassaolo
if 'DataLoader' not in locals(): raise NameError("DataLoader ei ole määritelty (tuo torch.utils.data).")
if 'nn' not in locals() or 'optim' not in locals(): raise NameError("torch.nn tai torch.optim ei ole tuotu.")

def train_model(model, train_loader, valid_loader, criterion, optimizer, epochs, device, patience):
    """Kouluttaa mallin ja käyttää Early Stoppingia (sis. .squeeze(-1) korjauksen)."""

    # Alkutarkistukset
    if not isinstance(model, nn.Module): raise TypeError("Vaaditaan PyTorch-malli.")
    if not isinstance(train_loader, DataLoader): raise TypeError("Vaaditaan DataLoader train_loaderille.")
    if not isinstance(valid_loader, DataLoader): raise TypeError("Vaaditaan DataLoader valid_loaderille.")
    if not isinstance(criterion, nn.modules.loss._Loss): raise TypeError("Vaaditaan PyTorch häviöfunktio.")
    if not isinstance(optimizer, optim.Optimizer): raise TypeError("Vaaditaan PyTorch optimoija.")
    if not isinstance(epochs, int) or epochs <= 0: raise ValueError("Epochs oltava positiivinen kokonaisluku.")
    if not isinstance(patience, int) or patience <= 0: raise ValueError("Patience oltava positiivinen kokonaisluku.")


    train_losses = []
    valid_losses = []
    best_valid_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None # Tähän tallennetaan parhaan mallin tila

    print(f"\nAloitetaan koulutus {epochs} epochilla...")
    print(f"Early stopping -kärsivällisyys: {patience} epochia.")

    for epoch in tqdm(range(epochs), desc="Epochs"):
        model.train() # Aseta malli koulutustilaan
        running_train_loss = 0.0
        batch_count = 0
        try: # Try-except epochin sisällä
            for inputs, targets_scaled in train_loader:
                batch_count += 1
                inputs, targets_scaled = inputs.to(device), targets_scaled.to(device)

                # Nollaa gradientit
                optimizer.zero_grad()

                # Eteenpäinsyöttö
                outputs_scaled = model(inputs) # Ennusteet, muoto (batch, 24)

                # --- Käsittele kohdemuoto (squeeze) ---
                if targets_scaled.ndim == outputs_scaled.ndim + 1 and targets_scaled.shape[-1] == 1:
                    targets_squeezed = targets_scaled.squeeze(-1) # Muoto (batch, 24)
                elif targets_scaled.shape == outputs_scaled.shape:
                    targets_squeezed = targets_scaled # Muodot täsmäävät jo
                else:
                     print(f"\nVIRHE Epoch {epoch+1}, Batch {batch_count} (Train): Muodot eivät täsmää loss-laskentaa varten!")
                     print(f"Output shape: {outputs_scaled.shape}, Target shape: {targets_scaled.shape}")
                     raise RuntimeError("Muodot eivät täsmää loss-laskentaa varten")

                # Lasketaan häviö
                loss = criterion(outputs_scaled, targets_squeezed)

                # Taaksepäinvienti ja optimointi
                loss.backward()
                optimizer.step()

                running_train_loss += loss.item() * inputs.size(0) # Kerro erän koolla painotusta varten

            # Laske keskimääräinen häviö koko epochille
            epoch_train_loss = running_train_loss / len(train_loader.dataset) if len(train_loader.dataset) > 0 else 0
            train_losses.append(epoch_train_loss)

        except Exception as e_train_epoch:
             print(f"\nVIRHE koulutusloopissa (Epoch {epoch+1}, Batch {batch_count}): {e_train_epoch}")
             traceback.print_exc()
             print("Keskeytetään koulutus.")
             # Palauta tähänastiset tulokset ja None mallille osoittamaan virhettä
             model = None # Merkitään malli vialliseksi
             return model, train_losses, valid_losses


        # --- Validointivaihe ---
        model.eval() # Aseta malli evaluointitilaan
        running_valid_loss = 0.0
        valid_batch_count = 0
        try: # Try-except validointiin
            with torch.no_grad(): # Ei lasketa gradientteja validoinnissa
                for inputs, targets_scaled in valid_loader:
                    valid_batch_count += 1
                    inputs, targets_scaled = inputs.to(device), targets_scaled.to(device)
                    outputs_scaled = model(inputs)

                    # --- Käsittele kohdemuoto (squeeze) ---
                    if targets_scaled.ndim == outputs_scaled.ndim + 1 and targets_scaled.shape[-1] == 1:
                        targets_squeezed = targets_scaled.squeeze(-1)
                    elif targets_scaled.shape == outputs_scaled.shape:
                        targets_squeezed = targets_scaled
                    else:
                         print(f"\nVIRHE Epoch {epoch+1} (Valid), Batch {valid_batch_count}: Muodot eivät täsmää loss-laskentaa varten!")
                         print(f"Output shape: {outputs_scaled.shape}, Target shape: {targets_scaled.shape}")
                         raise RuntimeError("Muodot eivät täsmää validointi loss-laskentaa varten")

                    loss = criterion(outputs_scaled, targets_squeezed)
                    running_valid_loss += loss.item() * inputs.size(0)

            epoch_valid_loss = running_valid_loss / len(valid_loader.dataset) if len(valid_loader.dataset) > 0 else 0
            valid_losses.append(epoch_valid_loss)

            print(f"Epoch {epoch+1:02d}/{epochs} - Train Loss: {epoch_train_loss:.6f} - Valid Loss: {epoch_valid_loss:.6f}", end="")

            # Early Stopping Check
            if epoch_valid_loss < best_valid_loss:
                best_valid_loss = epoch_valid_loss
                epochs_no_improve = 0
                try: # Tallenna paras malli
                    best_model_state = copy.deepcopy(model.state_dict())
                    print(" (Uusi paras!)")
                except Exception as e_state:
                     print(f" (VIRHE mallin tilan tallennuksessa: {e_state})")
                     best_model_state = None # Älä käytä viallista tilaa
            else:
                epochs_no_improve += 1
                print(f" (Ei parannusta {epochs_no_improve}/{patience})")

            if epochs_no_improve >= patience:
                print(f"\nEarly stopping {patience} epochin jälkeen ilman parannusta.")
                break # Pysäytä koulutusloop

        except Exception as e_valid_epoch:
             print(f"\nVIRHE validointiloopissa (Epoch {epoch+1}, Batch {valid_batch_count}): {e_valid_epoch}")
             traceback.print_exc()
             print("Keskeytetään koulutus.")
             model = None # Merkitään malli vialliseksi
             return model, train_losses, valid_losses


    # --- Koulutusloopin jälkeen ---
    # Palauta paras löydetty mallin tila (jos löytyi ja tallennus onnistui)
    if best_model_state:
         print("\nLadataan paras malli Early Stoppingin perusteella.")
         try:
              model.load_state_dict(best_model_state)
         except Exception as e_load:
              print(f"VIRHE parhaan mallin tilan latauksessa: {e_load}. Jatketaan viimeisimmällä mallilla.")
    elif epochs > 0 and train_losses is not None : # Jos ei early stopping, mutta ajettiin onnistuneesti
         print("\nKoulutus päättyi ilman Early Stoppingia tai paras tila viallinen. Käytetään viimeisintä mallia.")
    else: # Jos ei ajettu yhtään epochia tai koulutus keskeytyi heti
         print("\nKoulutusta ei ajettu loppuun / keskeytyi. Malli saattaa olla alustamaton.")


    return model, train_losses, valid_losses

print("\nOsa 5: Koulutusfunktion Määrittely (GRU v3) - OK")

In [ ]:
# @title 6. Mallin Koulutus (Suoritus) (GRU v3)

import torch.nn as nn # Varmistetaan tuonnit
import torch.optim as optim # Varmistetaan tuonnit
import matplotlib.pyplot as plt # Varmistetaan tuonnit
import traceback # Varmistetaan tuonti

print("--- Aloitetaan Mallin Koulutus (Suoritus) ---")

# Alustetaan model Noneksi siltä varalta että koulutus ei käynnisty tai epäonnistuu
model = None
train_losses = None
valid_losses = None

# Varmistetaan, että kaikki tarvittavat muuttujat edellisistä osista ovat olemassa
required_vars_exist = True
vars_to_check = ['train_loader', 'valid_loader', 'INPUT_SIZE', 'device',
                 'HIDDEN_SIZE', 'NUM_LAYERS', 'OUTPUT_SIZE', 'DROPOUT_PROB',
                 'LEARNING_RATE', 'EPOCHS', 'EARLY_STOPPING_PATIENCE', 'GRUModel', 'train_model']
missing_vars = []
for var in vars_to_check:
    # Tarkista sekä olemassaolo että ettei arvo ole None (paitsi mahdollisesti train_losses/valid_losses)
    if var not in locals() or (locals()[var] is None and var not in ['train_losses', 'valid_losses']):
        missing_vars.append(var)
        required_vars_exist = False

if required_vars_exist:
    # --- Mallin, häviöfunktion ja optimoijan alustus ---
    if isinstance(INPUT_SIZE, int) and INPUT_SIZE > 0:
        try:
            model = GRUModel(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, OUTPUT_SIZE, DROPOUT_PROB).to(device)
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

            print("\n--- GRU-Malli (ennen koulutusta) ---")
            print(model)
            print(f"Ominaisuuksien määrä (Input size): {INPUT_SIZE}")
            param_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
            print(f"Koulutettavien parametrien määrä: {param_count}")
            print(f"Käytettävä laite: {device}")
            print("-------------------------------------\n")

            # --- Koulutetaan malli ---
            try:
                model, train_losses, valid_losses = train_model(
                    model, train_loader, valid_loader, criterion, optimizer, EPOCHS, device, EARLY_STOPPING_PATIENCE
                )

                # Tarkistetaan paluuarvot
                if model is None or train_losses is None or valid_losses is None:
                     print("\nKoulutus keskeytyi tai epäonnistui train_model-funktiossa.")
                     model = None # Varmistetaan Noneksi
                else:
                     print("\nKoulutus suoritettu.")

                     # --- Piirretään häviökäyrät (vain jos koulutus onnistui ja listat ok) ---
                     if train_losses and valid_losses: # Varmista että listat eivät ole tyhjiä tai None
                        plt.figure(figsize=(10, 5))
                        plt.plot(train_losses, label='Training Loss')
                        plt.plot(valid_losses, label='Validation Loss')
                        plt.title('Training and Validation Loss')
                        plt.xlabel('Epoch')
                        plt.ylabel('Loss (MSE)')
                        # Yritä käyttää log-skaalaa vain jos arvot sen sallivat
                        try:
                            min_loss_val = min(min(train_losses, default=1.0), min(valid_losses, default=1.0))
                            if min_loss_val > 1e-9: # Tarkistus > 0
                               plt.yscale('log')
                               print("Käytetään logaritmista skaalaa häviökuvaajassa.")
                            else:
                               print("Käytetään lineaarista skaalaa häviökuvaajassa (min loss <= 0).")
                        except: # Jos minimin laskenta epäonnistuu tms.
                            print("Käytetään lineaarista skaalaa häviökuvaajassa.")
                            plt.yscale('linear')

                        plt.legend()
                        plt.grid(True, linestyle=':', alpha=0.7)
                        plt.show()
                     elif train_losses is not None and valid_losses is not None:
                          print("Häviölistat ovat tyhjiä, ei voida piirtää kuvaajaa.")
                     else:
                          print("Häviölistoja ei saatu koulutuksesta, ei voida piirtää kuvaajaa.")


            except Exception as e:
                print(f"\nVIRHE train_model-kutsun aikana: {e}")
                traceback.print_exc()
                model = None # Aseta malli Noneksi

        except Exception as e_init:
             print(f"\nVIRHE mallin, criterionin tai optimizerin alustuksessa: {e_init}")
             traceback.print_exc()
             model = None # Aseta malli Noneksi

    else:
        print(f"\nKoulutusta ei aloiteta, virheellinen INPUT_SIZE: {INPUT_SIZE}")
        model = None # Estetään jatko

else:
    print(f"\nKoulutusta ei voida aloittaa, koska yksi tai useampi tarvittava muuttuja puuttuu: {missing_vars}")
    model = None # Estetään jatko

# Tulostetaan lopputilanne
if model is not None:
     print("\nOsa 6: Koulutuksen suoritus - VALMIS")
else:
     print("\nOsa 6: Koulutuksen suoritus - EPÄONNISTUI / OHITETTIIN")

In [ ]:
# @title 7. Arviointifunktioiden Määrittely (GRU v3)

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler # Varmistetaan tuonti
import torch # Varmistetaan tuonti
from torch.utils.data import DataLoader # Varmistetaan tuonti
from tqdm.notebook import tqdm # Varmistetaan tuonti
import traceback

print("--- Määritellään arviointifunktiot ---")

# Funktio baseline-ennusteen laskemiseen
# Käytetään yksinkertaista persistenssiä: ennustetaan koko 24h jaksolle
# sama arvo kuin TODELLINEN arvo jakson ensimmäisellä tunnilla.
def calculate_baseline_persistence_pytorch(targets_original_np, prediction_horizon):
    """Laskee naiivin persistenssi-baselinen PyTorch-datarakenteelle."""
    print("Lasketaan Baseline-ennuste (jakson eka arvo toistuu)...")
    try:
        # targets_original_np muoto: (samples, horizon, 1)
        if targets_original_np.ndim != 3 or targets_original_np.shape[-1] != 1:
             raise ValueError(f"Odotettiin 3D-muotoa (samples, horizon, 1), saatiin {targets_original_np.shape}")
        if targets_original_np.shape[0] == 0:
             return np.array([]) # Palauta tyhjä, jos ei näytteitä

        # Otetaan ensimmäisen tunnin arvo (indeksi 0) jokaisesta näytteestä
        first_vals = targets_original_np[:, 0, 0] # Muoto: (samples,)
        # Muotoillaan (samples, 1) repeatia varten
        first_vals_for_repeat = first_vals[:, np.newaxis]

        # Toistetaan ensimmäinen arvo koko ennustehorisontille
        baseline_preds = np.repeat(first_vals_for_repeat, prediction_horizon, axis=1) # Muoto (samples, horizon)
        return baseline_preds
    except Exception as e_base:
        print(f"VIRHE baseline-laskennassa: {e_base}")
        traceback.print_exc()
        # Yritetään palauttaa oikean muotoisia nollia
        try:
            return np.zeros((targets_original_np.shape[0], prediction_horizon))
        except: return None


# Funktio mallin suorituskyvyn arviointiin
def evaluate_model_performance_pytorch(model, test_loader, device, o3_scaler, o3_threshold_8h, prediction_horizon):
    """Arvioi PyTorch-mallia testidatalla, laskee metriikat ja vertaa baselineen."""
    print("\n--- evaluate_model_performance_pytorch -funktion suoritus alkaa ---")

    # Alkutarkistukset
    if model is None: print("evaluate_model_performance_pytorch: Malli puuttuu."); return None, None
    if test_loader is None: print("evaluate_model_performance_pytorch: Testilataaja puuttuu."); return None, None
    if o3_scaler is None: print("evaluate_model_performance_pytorch: O3-skaalain puuttuu."); return None, None
    if not isinstance(o3_scaler, StandardScaler): print("VAROITUS: o3_scaler ei ole StandardScaler."); # Jatketaan silti

    model.eval() # Aseta malli arviointitilaan
    all_preds_orig_list = []
    all_targets_orig_list = []

    print("Aloitetaan ennusteiden tekeminen testidatalla...")
    try: # Try-except ennustusloopin ympärille
        with torch.no_grad():
            # test_loader tuottaa (inputs, targets_original_batch)
            for inputs, targets_original_batch in tqdm(test_loader, desc="Testaus (evaluate)"):
                inputs = inputs.to(device)
                # targets_original_batch on jo oikeassa muodossa (batch, horizon, 1)

                # Tee ennuste mallilla
                outputs_scaled = model(inputs) # Malli tuottaa skaalattuja ennusteita, muoto (batch, horizon)

                # Käännä ennusteiden skaalaus
                preds_scaled_np = outputs_scaled.cpu().numpy()
                # Skaalain odottaa 2D-muotoa (n_samples * n_features, 1)
                preds_reshaped = preds_scaled_np.reshape(-1, 1)
                preds_orig_np = o3_scaler.inverse_transform(preds_reshaped).reshape(preds_scaled_np.shape) # Palauta muotoon (batch, horizon)

                # Kerää ennusteet ja alkuperäiset kohteet listoihin
                all_preds_orig_list.append(preds_orig_np)
                all_targets_orig_list.append(targets_original_batch.cpu().numpy()) # Muoto (batch, horizon, 1)

        print("Ennusteiden tekeminen ja kerääminen valmis.")

        # Yhdistä kaikki erät yhdeksi isoksi numpy arrayksi
        if not all_preds_orig_list or not all_targets_orig_list:
             raise ValueError("Ennusteiden tai kohteiden keräys epäonnistui (listat tyhjiä).")

        all_preds_orig = np.concatenate(all_preds_orig_list, axis=0) # Muoto (total_samples, horizon)
        all_targets_original = np.concatenate(all_targets_orig_list, axis=0) # Muoto (total_samples, horizon, 1)

        print(f"Kerätty {all_preds_orig.shape[0]} ennustetta/kohdetta.")
        print(f"all_preds_orig muoto: {all_preds_orig.shape}, all_targets_original muoto: {all_targets_original.shape}")

        # Poista viimeinen dimensio kohteista metriikoita varten
        if all_targets_original.ndim == 3 and all_targets_original.shape[-1] == 1:
            targets_eval = all_targets_original.squeeze(-1) # Muoto (total_samples, horizon)
            print(f"Muutettu kohdemuoto metriikoita varten: {targets_eval.shape}")
        elif all_targets_original.ndim == 2:
             targets_eval = all_targets_original # Oletetaan jo oikea muoto
             print(f"Kohdemuoto oli jo 2D: {targets_eval.shape}")
        else:
             raise ValueError(f"Odottamaton kohdemuoto metriikoille: {all_targets_original.shape}")

        # Varmistetaan muodot vielä kerran
        if all_preds_orig.shape != targets_eval.shape:
             raise ValueError(f"Lopulliset muodot eivät täsmää: Pred={all_preds_orig.shape}, Target={targets_eval.shape}")

        # --- Laske regressiometriikat (GRU) ---
        print("\nLasketaan GRU-mallin regressiometriikat...")
        # Lasketaan virhe kaikista ennustepisteistä (samples * horizon)
        rmse_gru = np.sqrt(mean_squared_error(targets_eval.ravel(), all_preds_orig.ravel()))
        mae_gru = mean_absolute_error(targets_eval.ravel(), all_preds_orig.ravel())
        print(f"\n--- GRU-Mallin Arviointi (kaikki {prediction_horizon} tuntia) ---")
        print(f"RMSE: {rmse_gru:.4f} µg/m³")
        print(f"MAE:  {mae_gru:.4f} µg/m³")

        # --- Laske Baseline ---
        baseline_preds = calculate_baseline_persistence_pytorch(all_targets_original, prediction_horizon) # Käyttää 3D-muotoa sisäisesti
        if baseline_preds is None or baseline_preds.shape != targets_eval.shape:
             print("VIRHE: Baseline-laskenta epäonnistui tai muoto väärä.")
             rmse_baseline, mae_baseline = None, None
        else:
            # --- Laske regressiometriikat (Baseline) ---
            print("\nLasketaan Baseline-mallin regressiometriikat...")
            rmse_baseline = np.sqrt(mean_squared_error(targets_eval.ravel(), baseline_preds.ravel()))
            mae_baseline = mean_absolute_error(targets_eval.ravel(), baseline_preds.ravel())
            print(f"\n--- Baseline-Mallin Arviointi (Naiivi Persistenssi) ---")
            print(f"RMSE: {rmse_baseline:.4f} µg/m³")
            print(f"MAE:  {mae_baseline:.4f} µg/m³")

            # --- Vertailu ---
            print("\n--- Vertailu Baselineen ---")
            if rmse_gru is not None and rmse_baseline is not None:
                improvement_rmse = rmse_baseline - rmse_gru
                print(f"GRU vs Baseline RMSE: {improvement_rmse:+.4f} µg/m³ ({'GRU parempi' if improvement_rmse > 0 else 'Baseline parempi tai sama'})")
            else: print("RMSE-vertailua ei voida tehdä.")
            if mae_gru is not None and mae_baseline is not None:
                improvement_mae = mae_baseline - mae_gru
                print(f"GRU vs Baseline MAE:  {improvement_mae:+.4f} µg/m³ ({'GRU parempi' if improvement_mae > 0 else 'Baseline parempi tai sama'})")
            else: print("MAE-vertailua ei voida tehdä.")

        # --- 8h Liukuvan keskiarvon arviointi ---
        # Lasketaan kullekin 24h jaksolle erikseen
        warnings_actual = []
        warnings_pred = []
        n_samples_total = all_preds_orig.shape[0]
        print(f"\nLasketaan 8h liukuvia keskiarvoja {n_samples_total} jaksolle ja verrataan kynnysarvoon ({o3_threshold_8h} µg/m³)...")

        for i in range(n_samples_total):
            actual_24h = targets_eval[i, :] # Muoto (horizon,)
            pred_24h = all_preds_orig[i, :]  # Muoto (horizon,)

            actual_series = pd.Series(actual_24h)
            pred_series = pd.Series(pred_24h)

            actual_8h_avg = actual_series.rolling(window=8, min_periods=1).mean()
            pred_8h_avg = pred_series.rolling(window=8, min_periods=1).mean()

            actual_warning_triggered = actual_8h_avg.max() > o3_threshold_8h
            pred_warning_triggered = pred_8h_avg.max() > o3_threshold_8h

            warnings_actual.append(actual_warning_triggered)
            warnings_pred.append(pred_warning_triggered)

        warnings_actual = np.array(warnings_actual)
        warnings_pred = np.array(warnings_pred)
        print("Liukuvien keskiarvojen laskenta valmis.")

        # --- Tulosta 8h varoitusmetriikat ---
        print("\n--- 8h Liukuvan Keskiarvon Varoitustason Ylityksen Arviointi (GRU - Jaksoittain) ---")
        print(f"Todellisia varoitusjaksoja testidatassa (> {o3_threshold_8h} µg/m³): {warnings_actual.sum()} / {n_samples_total}")
        print(f"Ennustettuja varoitusjaksoja (GRU):                      {warnings_pred.sum()} / {n_samples_total}")

        if warnings_actual.sum() > 0 or warnings_pred.sum() > 0:
            print("\nSekaannusmatriisi (Confusion Matrix) varoituksille (GRU - Jaksoittain):")
            cm = confusion_matrix(warnings_actual, warnings_pred, labels=[False, True])
            cm_df = pd.DataFrame(cm, index=['Todellinen EI Varoitusta', 'Todellinen KYLLÄ Varoitus'],
                               columns=['Ennuste EI', 'Ennuste KYLLÄ'])
            print(cm_df)
            print("\nLuokitteluraportti varoituksille (GRU - Jaksoittain):")
            report = classification_report(warnings_actual, warnings_pred, target_names=['Ei Varoitusta', 'Varoitus'], labels=[False, True], zero_division=0)
            print(report)
            TN, FP, FN, TP = cm.ravel()
            recall_warning = TP / (TP + FN) if (TP + FN) > 0 else 0
            precision_warning = TP / (TP + FP) if (TP + FP) > 0 else 0
            print(f"\n---> TÄRKEIMMÄT VAROITUSMETRIIKAT (Jaksoittain):")
            print(f"  Recall (Herkkyys) 'Varoitus'-luokalle: {recall_warning:.4f}")
            print(f"  Precision (Tarkkuus) 'Varoitus'-luokalle: {precision_warning:.4f}")
        else:
            print(f"\nEi todellisia eikä ennustettuja varoitusjaksoja testidatassa kynnysarvolla {o3_threshold_8h} µg/m³.")


        print("\n--- evaluate_model_performance_pytorch -funktion suoritus päättyi onnistuneesti ---")
        # Palauta alkuperäiset ennusteet ja kohteet (muokattu 2D-muotoon)
        return all_preds_orig, targets_eval # Palautetaan 2D muodot

    except Exception as e:
        print(f"\n-----> VIRHE evaluate_model_performance_pytorch -FUNKTIOSSA <-----")
        print(f"Virhetyyppi: {type(e).__name__}")
        print(f"Virheilmoitus: {e}")
        print("Traceback:")
        traceback.print_exc()
        print("---------------------------------------------------------")
        print("Palautetaan None, None, koska arviointi epäonnistui.")
        return None, None # Palauta None virhetilanteessa


print("\nOsa 7: Arviointifunktioiden Määrittely (GRU v3) - OK")

In [ ]:
# @title 8. Mallin Arviointi (Suoritus) (GRU v3)

import traceback
import numpy as np # Varmistetaan tuonti
import pandas as pd # Varmistetaan tuonti

print("--- Aloitetaan Mallin Arviointi (Suoritus) ---")

# Alustetaan tulosmuuttujat Noneksi
test_preds_orig_gru = None
test_targets_orig_gru = None
evaluation_gru_successful = False # Lipuke onnistumiselle

# Varmistetaan, että kaikki tarvittavat muuttujat edellisistä osista ovat olemassa
required_eval_vars = ['model', 'test_loader', 'device', 'o3_scaler',
                      'O3_THRESHOLD_8H_AVG', 'PREDICTION_HORIZON',
                      'evaluate_model_performance_pytorch'] # Tarkista myös funktion olemassaolo
missing_eval_vars = []
for var in required_eval_vars:
     # Tarkista olemassaolo JA ettei arvo ole None (paitsi model voi olla None jos koulutus epäonnistui)
     if var not in locals() or (locals()[var] is None and var != 'model'):
          missing_eval_vars.append(var)

# Tarkistetaan erikseen, onko malli None (koulutus epäonnistui?)
if 'model' not in locals() or model is None:
     missing_eval_vars.append('model (ei koulutettu/epäonnistui)')

if not missing_eval_vars:
    # --- Suoritetaan arviointi ---
    try:
        print("\nKutsutaan evaluate_model_performance_pytorch...")
        # Huom: Funktio palauttaa ennusteet ja kohteet 2D-muodossa (samples, horizon)
        test_preds_orig_gru, test_targets_orig_gru = evaluate_model_performance_pytorch(
            model,
            test_loader,
            device,
            o3_scaler,
            O3_THRESHOLD_8H_AVG, # Käyttää Osa 1:ssä määriteltyä (tai Osa 9:ssä muokattua, jos tämä olisi ARIMA-notebook)
            PREDICTION_HORIZON
        )

        # Tarkistetaan paluuarvot
        if test_preds_orig_gru is not None and test_targets_orig_gru is not None:
            print("\nArviointifunktion ajo suoritettu.")
            # Tarkistetaan vielä tyypit ja muodot varmuuden vuoksi
            if isinstance(test_preds_orig_gru, np.ndarray) and isinstance(test_targets_orig_gru, np.ndarray) \
               and test_preds_orig_gru.ndim == 2 and test_targets_orig_gru.ndim == 2 \
               and test_preds_orig_gru.shape == test_targets_orig_gru.shape:
                 print(f"Arviointi palautti validit tulokset muodoissa: Preds={test_preds_orig_gru.shape}, Targets={test_targets_orig_gru.shape}")
                 evaluation_gru_successful = True
                 # Tallennetaan tulokset myöhempää käyttöä varten (ne ovat jo muuttujissa)
            else:
                 print("VIRHE: Arviointifunktion palauttamat arvot eivät ole odotettuja numpy arrayta tai muodot eivät täsmää.")
                 test_preds_orig_gru, test_targets_orig_gru = None, None # Nollataan virheelliset tulokset

        else:
            print("\nArviointi epäonnistui (evaluate_model_performance_pytorch palautti None).")
            # Muuttujat ovat jo None

    except Exception as e:
        print(f"\nVIRHE arvioinnin suorituksessa: {e}")
        traceback.print_exc()
        # Varmistetaan, että muuttujat ovat None virheen sattuessa
        test_preds_orig_gru, test_targets_orig_gru = None, None

else:
    print(f"\nArviointia ei voida suorittaa, koska yksi tai useampi tarvittava muuttuja puuttuu tai on None: {missing_vars}")
    # Varmistetaan, että muuttujat ovat None
    test_preds_orig_gru, test_targets_orig_gru = None, None


# Tulostetaan lopputilanne
if evaluation_gru_successful:
     print("\nOsa 8: Arvioinnin suoritus (GRU v3) - VALMIS")
else:
     print("\nOsa 8: Arvioinnin suoritus (GRU v3) - EPÄONNISTUI / OHITETTIIN")